This is stage 2 for developing a discrete MDP wrapper for the highway environment based on discretizedMDP.ipynb.

The purpose of this code is to implement multiprocessing and set up a separate discretized mdp python script in the codebase.

In [1]:
# Move up one directory level to import local instance of highway environment
%cd ..
%pwd

d:\WorkFiles\ProjectCode\ActiveProjects\HoLab\Mine\HighwayEnv-TRI


'd:\\WorkFiles\\ProjectCode\\ActiveProjects\\HoLab\\Mine\\HighwayEnv-TRI'

Import necessary modules

In [2]:
from highway_env.envs.discreteMDP_wrapper import HighwayDiscreteMDP, OptimalPolicy

import cProfile
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)    # Other options: INFO, WARNING, ERROR, CRITICAL

import copy

<frozen importlib._bootstrap>:488: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


Set up MDP

In [3]:
config = {
        "observation": {
            "type": "Kinematics",
            "vehicles_count": 50,
            "features": ["presence", "x", "y", "vx", "vy", "heading"],
            "normalize": False,
            "absolute": True,
            "order": "sorted",
            "observe_intentions": False,
            "include_obstacles": True
            }
        }
# highway_mdp = HighwayDiscreteMDP('highway-v0', config=config, render_mode='rgb_array')
# highway_mdp = HighwayDiscreteMDP('highway-v0', config=config, render_mode=None)
# highway_mdp = HighwayDiscreteMDP('highway-fast-v0', config=config, render_mode=None)
highway_mdp = HighwayDiscreteMDP('highway-icy-fast-v0', config=config, render_mode=None)

c:\Users\Sounak\anaconda3\envs\myHighwayEnv\Lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [5]:
# TODO: Fix bug where executing 'populate_MDPtable' takes longer in consecutive runs
highway_mdp.populate_MDPtable(max_depth = 5)

INFO:root:Current Depth: 0 | Frontier: 5 | Visited: 1 | Transitions:5
INFO:root:Current Depth: 1 | Frontier: 9 | Visited: 2 | Transitions:10
INFO:root:Current Depth: 1 | Frontier: 13 | Visited: 3 | Transitions:15
INFO:root:Current Depth: 2 | Frontier: 17 | Visited: 4 | Transitions:20
INFO:root:Current Depth: 2 | Frontier: 21 | Visited: 5 | Transitions:25
INFO:root:Current Depth: 2 | Frontier: 25 | Visited: 6 | Transitions:30
INFO:root:Current Depth: 1 | Frontier: 29 | Visited: 7 | Transitions:35
INFO:root:Current Depth: 3 | Frontier: 33 | Visited: 8 | Transitions:40
INFO:root:Current Depth: 2 | Frontier: 37 | Visited: 9 | Transitions:45
INFO:root:Current Depth: 1 | Frontier: 41 | Visited: 10 | Transitions:50
INFO:root:Current Depth: 3 | Frontier: 45 | Visited: 11 | Transitions:55
INFO:root:Current Depth: 4 | Frontier: 49 | Visited: 12 | Transitions:60
INFO:root:Current Depth: 5 | Frontier: 48 | Visited: 13 | Transitions:60
INFO:root:Current Depth: 3 | Frontier: 52 | Visited: 14 | Trans

In [4]:
highway_mdp.populate_MDPtable(max_depth = 3)
matMDP = highway_mdp.get_MDPmatrices()
plan = highway_mdp.value_iteration()

In [5]:
policy = OptimalPolicy(highway_mdp)
acts = policy(highway_mdp.initial_state)

INFO:root:Action values: [(0, 2.6790451619094733), (1, 2.7467961636381086), (2, 2.6479067445813813), (3, 2.879825009203372), (4, 2.482746188728289)]


In [6]:
acts

[3]

In [ ]:
list(highway_mdp.currMDP.state_list)[0]

Construals Logic

TODO:
1. Create a m-vehicle n-icesheet environment
2. Split it into x=m+n+1 separate environments each with a single ice-sheet or vehicle or completely empty
3. Calculate the optimal policy for each of the x environments
4. Run each of those poliies in the oriinal m-vehicle n-icesheet environment.

In [11]:
construals = highway_mdp.get_construals_singleobj()

In [5]:
for constr in construals:
    # print(constr.env.unwrapped.road.vehicles)
    # print(constr.env.unwrapped.road.objects)
    # print("----------------")
    constr.populate_MDPtable(max_depth = 3)
    break
    

In [ ]:
construals[1].current_state

In [10]:
construals[1].populate_MDPtable(max_depth = 3)

INFO:root:Current Depth: 3 | Frontier: 20 | Visited: 4 | Transitions:15
INFO:root:Current Depth: 3 | Frontier: 16 | Visited: 4 | Transitions:15
INFO:root:Current Depth: 3 | Frontier: 11 | Visited: 4 | Transitions:15
INFO:root:Current Depth: 3 | Frontier: 6 | Visited: 4 | Transitions:15
INFO:root:Current Depth: 3 | Frontier: 0 | Visited: 4 | Transitions:15


In [15]:
constr

<OrderEnforcing<PassiveEnvChecker<HighwayEnvIcyFast<highway-icy-fast-v0>>>>

In [ ]:

matMDP = highway_mdp.get_MDPmatrices()
plan = highway_mdp.value_iteration()
policy = OptimalPolicy(highway_mdp)
acts = policy(highway_mdp.initial_state)

Print MDP Tables

In [5]:
print(highway_mdp.env.unwrapped.action_type.actions_indexes)

highway_mdp.currMDP.transition

{'LANE_LEFT': 0, 'IDLE': 1, 'LANE_RIGHT': 2, 'FASTER': 3, 'SLOWER': 4}


{(frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  0): {frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  1): {frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  2): {frozendict.frozendict({'position': (178.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  3): {frozendict.frozendict({'position': (181.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  4): {frozendict.frozendict({'position': (176.23, 0.0), 'speed': (20.66, 0.0), 'heading': 0.0}): 1},
 (frozendict.frozendict({'pos

In [6]:
highway_mdp.currMDP.reward

{(frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  0,
  frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0})): 0.7999999999999999,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  1,
  frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0})): 0.7999999999999999,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  2,
  frozendict.frozendict({'position': (178.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043})): 0.8327113850086175,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  3,
  frozendict.frozendict({'position': (181.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0})): 0.9157750342935529,
 (frozendict.frozendict({'position': (153.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}),
  4,
  frozendict.frozendict({'position': (176.23, 0.0), 'speed

In [7]:
highway_mdp.currMDP.absorption

{frozendict.frozendict({'position': (178.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (178.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043}): False,
 frozendict.frozendict({'position': (181.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (176.23, 0.0), 'speed': (20.66, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (196.57, 0.0), 'speed': (20.09, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (196.19, 3.6), 'speed': (20.05, 1.16), 'heading': 0.058}): False,
 frozendict.frozendict({'position': (198.96, 0.0), 'speed': (24.43, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (203.62, 0.0), 'speed': (25.0, 0.0), 'heading': 0.0}): False,
 frozendict.frozendict({'position': (203.3, 3.66), 'speed': (24.98, 1.08), 'heading': 0.043}): False,
 frozendict.frozendict({'position': (206.02, 0.0), 'speed': (29.34, 0.0), 'heading': 0.0}): False,
 fr

Find optimal policy

# TEST CODE

In [ ]:
import functools
import multiprocessing as mp
process_count = (mp.cpu_count()-2)
from pathos.multiprocessing import ProcessingPool as Pool    # Can run multiprocessing in interactive shell


max_depth = 2   # The number of steps to plan ahead
actions = highway_mdp.actions

start_env = (highway_mdp.initial_state, 0, highway_mdp)
visited = set()
transitions = {}
rewards = {}
absorption = {}
frontier = {start_env}
env2close = set()   # Maintain list of environments to close
while frontier:
    state, depth, curr_MDPstate = frontier.pop()
    visited.add(state)
    env2close.add(curr_MDPstate)
    if depth < max_depth:
        with Pool(process_count) as pool:
            return_vals = pool.map(functools.partial(HighwayDiscreteMDP.exeuteAction, envMDP=curr_MDPstate), actions)
        for action, next_state, trans_prob, reward, done, truncated, info, updated_MDPstate in return_vals:
            logging.debug(str(state[0]) + ' | ' + str(action) + ' | ' + str(next_state[0]))
            #2# Populate transitions table
            if (state[0], action) not in transitions:
                transitions[(state[0], action)] = {}
            if next_state[0] not in transitions[(state[0], action)]:
                transitions[(state[0], action)][next_state[0]] = trans_prob
            #2# Populate reward functions
            if (state[0], action) in rewards:
                rewards[(state[0], action, next_state[0])] += reward * trans_prob
            else:
                rewards[(state[0], action, next_state[0])] = reward * trans_prob
            #2# set absorption states
            absorption[next_state[0]] = done or truncated
            #2# Populate visited
            if next_state not in visited:
                frontier.add((next_state, depth + 1, updated_MDPstate))
            else:
                updated_MDPstate.env.close()
        MDPstatus = "Current Depth: " + str(depth) + " | Frontier: " + str(len(frontier)) +\
                    " | Visited: " + str(len(visited)) + " | Transitions:" + str(len(transitions))
        logging.info(MDPstatus)
env2close.discard(start_env[2]) #Keep the first step active
for curr_MDPstate in env2close:
    # close all duplicate environments
    curr_MDPstate.env.close()
env2close = set()   # Close ununsed environments at the end of the loop
# print(highway_mdp.actions)
highway_mdp.create_MDPTable(transition=transitions, absorption=absorption, reward=rewards, state_list=visited, action_list=actions)

INFO:root:Current Depth: 0 | Frontier: 5 | Visited: 1 | Transitions:5
INFO:root:Current Depth: 1 | Frontier: 9 | Visited: 2 | Transitions:10
INFO:root:Current Depth: 1 | Frontier: 13 | Visited: 3 | Transitions:15
INFO:root:Current Depth: 1 | Frontier: 16 | Visited: 5 | Transitions:20
INFO:root:Current Depth: 1 | Frontier: 13 | Visited: 12 | Transitions:25
INFO:root:Current Depth: 1 | Frontier: 4 | Visited: 17 | Transitions:25


In [ ]:
# Test differences in environment state for situations with same ego-vehivle state
visited = highway_mdp.populate_MDPtable(max_depth = 4)

for v1 in list(visited)[:-1]:
    for v2 in list(visited)[1:]:
        if v1 != v2 and v1[0] == v2[0]:
            print(v1[0],v2[0])
            for veh1, veh2 in zip(v1,v2):
                if veh1 != veh2:
                    print(veh1, veh2)
            print('\n')


frozendict.frozendict({'position': (251.43, 0.0), 'speed': (25.0, -0.0), 'heading': -0.0}) frozendict.frozendict({'position': (251.43, 0.0), 'speed': (25.0, -0.0), 'heading': -0.0})
frozendict.frozendict({'position': (250.74, 4.0), 'speed': (19.92, 0.01), 'heading': 0.0}) frozendict.frozendict({'position': (248.15, 3.95), 'speed': (19.14, 0.15), 'heading': 0.008})


frozendict.frozendict({'position': (253.83, 0.0), 'speed': (29.34, -0.0), 'heading': -0.0}) frozendict.frozendict({'position': (253.83, 0.0), 'speed': (29.34, -0.0), 'heading': -0.0})
frozendict.frozendict({'position': (248.15, 3.95), 'speed': (19.14, 0.15), 'heading': 0.008}) frozendict.frozendict({'position': (250.74, 4.0), 'speed': (19.92, 0.01), 'heading': 0.0})


frozendict.frozendict({'position': (244.38, 0.0), 'speed': (20.09, -0.0), 'heading': -0.0}) frozendict.frozendict({'position': (244.38, 0.0), 'speed': (20.09, -0.0), 'heading': -0.0})
frozendict.frozendict({'position': (248.15, 3.95), 'speed': (19.14, 0.15), '

In [ ]:
# |DEBUGGING CODE

print(highway_mdp.env.unwrapped.road.vehicles[0])
tmp = highway_mdp.env()
print(tmp.unwrapped.road.vehicles[0])
highway_mdp.step(1)
print(highway_mdp.env.unwrapped.road.vehicles[0])
print(tmp.unwrapped.road.vehicles[0])
highway_mdp.env = tmp
print(highway_mdp.env.unwrapped.road.vehicles[0])


# dir(highway_mdp.env.unwrapped)

next_state, reward, done, truncated, info = highway_mdp.step(1)
# next_state


MDPVehicle #544: [207.38302935   8.        ]
MDPVehicle #248: [207.38302935   8.        ]
1 [1.   1.   0.08 1.   0.   0.  ]
MDPVehicle #544: [232.38302935   8.        ]
MDPVehicle #248: [207.38302935   8.        ]
MDPVehicle #248: [207.38302935   8.        ]


In [ ]:
config = {
        "observation": {
            "type": "Kinematics",
            "vehicles_count": 50,
            "features": ["presence", "x", "y", "vx", "vy", "heading"],
            "normalize": False,
            "absolute": True,
            "order": "sorted",
            "observe_intentions": False,
            "include_obstacles": True
            }
        }

env = gym.make('highway-v0', config=config, render_mode='human')

In [75]:
vehicles = []
for veh_state in obs:
    feature_vals = {k: v for k,v in zip(config["observation"]["features"], veh_state)}
    veh = {}
    veh["position"] = tuple(np.round((feature_vals["x"],feature_vals["y"]), 2))
    veh["speed"] = tuple(np.round((feature_vals["vx"],feature_vals["vy"]), 2))
    veh["heading"] = np.round(feature_vals["heading"], 3)
    vehicles.append(frozendict(veh))
vehicles = tuple(vehicles)
vehicles

(frozendict.frozendict({'position': (1.0, 0.08), 'speed': (1.0, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.08, 0.0), 'speed': (-0.24, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.33, -0.04), 'speed': (-0.18, 0.0), 'heading': 0.003}),
 frozendict.frozendict({'position': (0.45, 0.04), 'speed': (-0.31, 0.0), 'heading': 0.005}),
 frozendict.frozendict({'position': (0.72, 0.0), 'speed': (-0.19, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.93, 0.04), 'speed': (-0.19, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (1.0, -0.08), 'speed': (-0.27, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (1.0, -0.04), 'speed': (-0.24, -0.0), 'heading': -0.0}),
 frozendict.frozendict({'position': (1.0, -0.0), 'speed': (-0.16, 0.0), 'heading': 0.001}),
 frozendict.frozendict({'position': (1.0, -0.08), 'speed': (-0.28, 0.0), 'heading': 0.0}),
 frozendict.frozendict({'position': (0.0, 0.0), 'speed': (0.0, 0.0), 'heading': 0.0}),


In [77]:
import sys

# sys.getsizeof(transitions)
sys.getsizeof(highway_mdp.copy_env())
# sys.getsizeof(int(10.0))


48